# L3 Probate Complaint Summary Demo

This is a break down of the notebook Develop/Notebooks/Layer_3/Optic/L3Optic_ProbateComplaintSummary<br/>
The Notebook can be found on, <br/>
https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary


## Import packages

In [41]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import *
import datetime
import decimal
from delta.tables import *
from pprint import pprint as pp

StatementMeta(misparkpool, 4907, 2, Finished, Available)

The next 4 cells run other Notebooks, importing all the classes defined in them.<br/> 
The location of those Notebooks is `Develop/Notebooks/Commons/`  (https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/commons_notebooks.png) 


In [42]:
%run Commons/DataSpecification

StatementMeta(, 4907, -1, Finished, Available)

In [43]:
%run Commons/SynapseDatabaseWriter

StatementMeta(, 4907, -1, Finished, Available)

In [44]:
%run Commons/SynapseDatabaseReader

StatementMeta(, 4907, -1, Finished, Available)

In [45]:
%run Commons/QA_Support

StatementMeta(, 4907, -1, Finished, Available)

## Parameters

Here the parameters are hard coded into the Notebook

In [46]:
CurrentExecutionId = 0
parametersJSON = json.dumps({
    "LastModifiedDate": "2021-02-10",
    "sourceName": "Optic",
    "processName": "L3Optic_ProbateComplaintSummary",
    "lakeAccountName": "mipolybasestagingtest",
    "lakeContainerName": "transformation",
    "tblLakeFolder": "Layer_1/XCutting/Optic",
    "tblLakeFileNameItem": "L1_Optic_tblOpticItem",
    "tblLakeFileNameItemStep": "L1_Optic_tblOpticItemStep",
    "tblLakeFileNamePayment": "L1_Optic_tblOpticPayment",
    "fctLakeFolder": "Layer_3/XCutting/Optic",
    "fctLakeFileName": "L3_Optic_fctProbateComplaintSummary",
    "fctLakeFileNameOpticComplaintSummary": "L3_Optic_fctOpticComplaintSummary",
    "dwhEnvironment": "test",
    "dwhDatabase": "mi_dwh_test",
    "dwhSchema": "L3_Optic",
    "dwhTableName": "fctProbateComplaintSummary",
    "lakeErrorFileName": "L3_Error_tblProbateComplaintSummary",
    "lakeErrorFolder": "Layer_3/Optic/ErrorTable",
    "lakeErrorTableName": "tblProbateComplaintSummary_Error",
    "expectationSuiteName": "L3/Optic"
})

print(f'When you use json.dumps the type for the json is {type(parametersJSON)}')

StatementMeta(misparkpool, 4907, 23, Finished, Available)

When you use json.dumps the type for the json is <class 'str'>

## Logging variables

In [47]:
#These two variables will be used to count
RowsRead = 0
RowsCopied = 0 

StatementMeta(misparkpool, 4907, 24, Finished, Available)

## Populate variables

Here we use the parsed_json dictionary to populate all relevant variables

In [48]:
parsed_json = json.loads(parametersJSON) #Turns the json above into a dictionary
print(f'Using json.loads turns the json in str type to a type {type(parsed_json)}')

# Run parameters
LastLoadedDateTime = parsed_json['LastModifiedDate']
sourceName = parsed_json['sourceName']
processName = parsed_json['processName']

# datalake storage details (target)
lakeAccountName = parsed_json['lakeAccountName']
lakeContainerName = parsed_json['lakeContainerName']

# fct (target)
fctLakeFolder = parsed_json['fctLakeFolder']
fctLakeFileName = parsed_json['fctLakeFileName']
fctTableNameOpticComplaintSummary = parsed_json['fctLakeFileNameOpticComplaintSummary']
# dwh (target - datawharehouse)
dwhEnvironment = parsed_json['dwhEnvironment']
dwhDatabase = parsed_json['dwhDatabase']
dwhSchema = parsed_json['dwhSchema']
dwhTableName = parsed_json['dwhTableName']

# QA
lakeErrorFileName = parsed_json['lakeErrorFileName']
lakeErrorFolder = parsed_json['lakeErrorFolder']
lakeErrorTableName = parsed_json['lakeErrorTableName']
expectationSuiteName = parsed_json['expectationSuiteName']
lakeErrorFilePath = 'abfss://{container}@{storage}.dfs.core.windows.net/{folder}/{file}.parquet'.format(
    container=lakeContainerName,
    storage=lakeAccountName,
    file=lakeErrorFileName,
    folder=lakeErrorFolder
)

print(f"""
LastLoadedDateTime: {LastLoadedDateTime}
sourceName: {sourceName} 
processName: {processName}""")

StatementMeta(misparkpool, 4907, 25, Finished, Available)

Using json.loads turns the json in str type to a type <class 'dict'>

LastLoadedDateTime: 2021-02-10
sourceName: Optic 
processName: L3Optic_ProbateComplaintSummary

## Connect to data lake

Connection is established via linked service.

When the linked service authentication method is set to Managed Identity or Service Principal, the linked service will use the Managed Identity or Service Principal token with the LinkedServiceBasedTokenProvider provider, for more details visit: <br/> 
https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-secure-credentials-with-tokenlibrary?pivots=programming-language-python

In [49]:
# Will give an error before the next cell is run, as there is no such configuration yet
spark.conf.get("spark.storage.synapse.linkedServiceName")

StatementMeta(misparkpool, 4907, 26, Finished, Available)

Py4JJavaError: An error occurred while calling o209.get.
: java.util.NoSuchElementException: spark.storage.synapse.linkedServiceName
	at org.apache.spark.sql.internal.SQLConf$$anonfun$getConfString$2.apply(SQLConf.scala:2605)
	at org.apache.spark.sql.internal.SQLConf$$anonfun$getConfString$2.apply(SQLConf.scala:2605)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:2605)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:74)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.Abstract

In [50]:
linked_service_name = 'LS_mi_persistent_storage' #Azure Blob Storage, see https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/linked_service.png
spark.conf.set("spark.storage.synapse.linkedServiceName", linked_service_name)
spark.conf.set("fs.azure.account.oauth.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider") #configuring authentication, see https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/linked_service_authentication.png

StatementMeta(misparkpool, 4907, 27, Finished, Available)

In [51]:
print(spark.conf.get("spark.storage.synapse.linkedServiceName"))
print(spark.conf.get("fs.azure.account.oauth.provider.type"))

StatementMeta(misparkpool, 4907, 28, Finished, Available)

LS_mi_persistent_storage
com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider

## Load schemas

From the `Commons/DataSpecification` notebook we have the following,

- A `DataSpec` is a light wrapper around a Spark schema that also keeps track of table details when provided. It can be loaded from a table DDL or from a JSON schema.
Create a spec based on a JSON schema, as created by the `json` function of Spark data types:
```python
mySpec = DataSpec.fromFile(
    DataIO.schemaLakeFilePath(storageAccount, 'path/to/mySchema.sql')
)
```
<br/><br/>
- The `DataIO` class includes a number of convenience functions to read and write data from the blob store and to the data lake. 
This function creates the full path of a schema file: 
```python
mySchemaPath = DataIO.schemaLakeFilePath(storageAccount, filePath)
```
<br/><br/>
- The `DataTransform` class makes it possible to transform a dataframe from one schema to another by renaming, casting fields and narrowing the selection down to the target fields, in the same order as defined in the target schema. You can use the schemas from a spec of a dataframe:
```python
trf = DataTransform.fromSchemas(sourceSpec.schema, targetSpec.schema)
targetDF = trf.transform(sourceDF)

trf = DataTransform.fromSchemas(sourceDF.schema, targetDF.schema)
targetDF = trf.transform(sourceDF)
```

In [52]:

fctProbateComplaintSummarySpec = DataSpec.fromFile(DataIO.schemaLakeFilePath(lakeAccountName, 'Layer_3/XCutting/Optic/fctProbateComplaintSummary.sql'))#Accessing the relevant schema, https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/probate_schema_path.png

print(f"""storageAccount = {lakeAccountName}
filePath = Layer_3/XCutting/Optic/fctProbateComplaintSummary.sql
URI = {DataIO.schemaLakeFilePath(lakeAccountName, 'Layer_3/XCutting/Optic/fctProbateComplaintSummary.sql')}

Schema = {fctProbateComplaintSummarySpec.schema}""")

StatementMeta(misparkpool, 4907, 29, Finished, Available)

storageAccount = mipolybasestagingtest
filePath = Layer_3/XCutting/Optic/fctProbateComplaintSummary.sql
URI = abfss://schemas@mipolybasestagingtest.dfs.core.windows.net/Layer_3/XCutting/Optic/fctProbateComplaintSummary.sql

Schema = StructType(List(StructField(DateKey,TimestampType,true),StructField(SubjectLocationKey,DecimalType(10,6),true),StructField(OwningLocationKey,DecimalType(10,6),true),StructField(CustomerInteractionTypeKey,DecimalType(10,6),true),StructField(InteractionReasonTypeKey,DecimalType(10,6),true),StructField(ChannelMethodTypeKey,DecimalType(10,6),true),StructField(SourceKey,DecimalType(10,6),true),StructField(InteractionCustomerTypeKey,DecimalType(10,6),true),StructField(ReceiptByStepCount,DecimalType(6,0),true),StructField(ReceiptsByItemCount,DecimalType(6,0),true),StructField(ConcludedByStepCount,DecimalType(6,0),true),StructField(ConcludedByItemCount,DecimalType(6,0),true),StructField(ConcludedAtFirstContactCount,DecimalType(6,0),true),StructField(ConcludedInTarg

## Load sources

In the class DataIO:
```python
def readReferenceData(filePath, format = 'delta'):
        return spark.read.format('delta').load(filePath)
```
```python
def lakeFilePath(container, storageAccount, folder, fileName):
        return f'abfss://{container}@{storageAccount}.dfs.core.windows.net/{folder}/{fileName}'
```
```python
def referenceDataLakeFilePath(storageAccount, filePath, container = None):
        if container is None:
            container = DataIO.CONTAINER_REFERENCE_DATA
        return f'abfss://{container}@{storageAccount}.dfs.core.windows.net/{filePath}'
```
```python
REFDATA_SHARED_PREFIX = 'Reporting_Reference'
```

In [53]:
#Creating data frames 
fctOpticComplaintSummary = spark.read.format('delta').load(
    DataIO.lakeFilePath(lakeContainerName, lakeAccountName, fctLakeFolder, f'{fctTableNameOpticComplaintSummary}.parquet')
) #The data source, https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/fctOpticComplaintSummary_data_source.png

dimJurisdiction = DataIO.readReferenceData(
    DataIO.referenceDataLakeFilePath(lakeAccountName, DataIO.REFDATA_SHARED_PREFIX+'/dimJurisdiction.parquet')
) #The data source, https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/dimJurisdiction_data_source.png

#Creating views of the df's using the pyspark.sql function createOrReplaceTempView
fctOpticComplaintSummary.createOrReplaceTempView('fctOpticComplaintSummary') 
dimJurisdiction.createOrReplaceTempView('dimJurisdiction') 

print(f"""fctOpticComplaintSummary (type = {type(fctOpticComplaintSummary)}):
container = {lakeContainerName}, 
storageAccount = {lakeAccountName},
folder = {fctLakeFolder},
fileName = {fctTableNameOpticComplaintSummary}.parquet
URI = {DataIO.lakeFilePath(lakeContainerName, lakeAccountName, fctLakeFolder, f'{fctTableNameOpticComplaintSummary}.parquet')}
fctOpticComplaintSummary Data Frame (total rows = {fctOpticComplaintSummary.count()})
""")
fctOpticComplaintSummary.show(n=2, vertical=True)


print(f"""dimJurisdiction (type = {type(dimJurisdiction)}):
storageAccount = {lakeAccountName}, 
filePath = Reporting_Reference/dimJurisdiction.parquet
URI = {DataIO.referenceDataLakeFilePath(lakeAccountName, DataIO.REFDATA_SHARED_PREFIX+'/dimJurisdiction.parquet')}
dimJurisdiction Data Frame (total rows = {dimJurisdiction.count()})
""")
dimJurisdiction.show(n=2)

StatementMeta(misparkpool, 4907, 30, Finished, Available)

fctOpticComplaintSummary (type = <class 'pyspark.sql.dataframe.DataFrame'>):
container = transformation, 
storageAccount = mipolybasestagingtest,
folder = Layer_3/XCutting/Optic,
fileName = L3_Optic_fctOpticComplaintSummary.parquet
URI = abfss://transformation@mipolybasestagingtest.dfs.core.windows.net/Layer_3/XCutting/Optic/L3_Optic_fctOpticComplaintSummary.parquet
fctOpticComplaintSummary Data Frame (total rows = 32)

-RECORD 0-------------------------------------------
 DateKey                      | 2022-04-07 00:00:00 
 OwningLocationKey            | 1.000302            
 SubjectLocationKey           | 1.000302            
 CustomerInteractionTypeKey   | 13.000004           
 InteractionReasonTypeKey     | 27.002233           
 ChannelMethodTypeKey         | 17.000006           
 SourceKey                    | 9.000014            
 JurisdictionKey              | 15.000001           
 InteractionCustomerTypeKey   | 33.000002           
 CaseSourceTypeKey            | null          

## Transform to FCT schema

In [65]:
# Create a raw dataframe from a SELECT

fctProbateComplaintSummaryRaw = spark.sql("""
SELECT f.DateKey DateKey
, f.OwningLocationKey OwningLocationKey
, f.SubjectLocationKey SubjectLocationKey
, f.CustomerInteractionTypeKey
, f.InteractionReasonTypeKey
, f.ChannelMethodTypeKey AS ChannelMethodTypeKey
, f.SourceKey SourceKey
, f.InteractionCustomerTypeKey AS InteractionCustomerTypeKey
, f.ReceiptByStepCount
, f.ReceiptsByItemCount
, f.ConcludedByStepCount
, f.ConcludedByItemCount
, f.ConcludedAtFirstContactCount
, f.ConcludedInTargetByStepCount
, f.DaysToConcludeCount
, f.UpheldCount
, f.PartUpheldCount
, f.NotUpheldCount
, f.ReviewCount
, f.AppealCount
, f.PaidAmount as AmountPaidAmount
, f.ComplaintHasPaymentCount
, f.OmbudsmanCaseCompletedCount     -- 9.0 DJ
FROM fctOpticComplaintSummary f
INNER JOIN dimJurisdiction d1
ON d1.JurisdictionKey = f.JurisdictionKey
WHERE d1.JurisdictionName =  'Probate'
""")

#use JurisdictionName =  'Civil Money Claims' to see the pipeline work

print(f'fctProbateComplaintSummaryRaw (type = {type(fctProbateComplaintSummaryRaw)}) Schema,')
fctProbateComplaintSummaryRaw.printSchema()

StatementMeta(misparkpool, 4907, 42, Finished, Available)

fctProbateComplaintSummaryRaw (type = <class 'pyspark.sql.dataframe.DataFrame'>) Schema,
root
 |-- DateKey: timestamp (nullable = true)
 |-- OwningLocationKey: decimal(10,6) (nullable = true)
 |-- SubjectLocationKey: decimal(10,6) (nullable = true)
 |-- CustomerInteractionTypeKey: decimal(10,6) (nullable = true)
 |-- InteractionReasonTypeKey: decimal(10,6) (nullable = true)
 |-- ChannelMethodTypeKey: decimal(10,6) (nullable = true)
 |-- SourceKey: decimal(10,6) (nullable = true)
 |-- InteractionCustomerTypeKey: decimal(10,6) (nullable = true)
 |-- ReceiptByStepCount: decimal(10,6) (nullable = true)
 |-- ReceiptsByItemCount: decimal(10,6) (nullable = true)
 |-- ConcludedByStepCount: decimal(10,6) (nullable = true)
 |-- ConcludedByItemCount: decimal(10,6) (nullable = true)
 |-- ConcludedAtFirstContactCount: decimal(6,0) (nullable = true)
 |-- ConcludedInTargetByStepCount: decimal(6,0) (nullable = true)
 |-- DaysToConcludeCount: decimal(9,0) (nullable = true)
 |-- UpheldCount: decimal(6,0

In [62]:
# Coerce it to the target schema

trfProbateComplaintSummary = DataTransform.fromSchemas(fctProbateComplaintSummaryRaw.schema, fctProbateComplaintSummarySpec.schema) #DataTransform class is from the Notebook Commons/DataSpecification

print(f"""2 schemas in fromSchemas,
fctProbateComplaintSummaryRaw (type = {type(fctProbateComplaintSummaryRaw)}):
{fctProbateComplaintSummaryRaw.schema}

fctProbateComplaintSummarySpec (type = {type(fctProbateComplaintSummarySpec)}):
{fctProbateComplaintSummarySpec.schema}
""")

fctProbateComplaintSummary = trfProbateComplaintSummary.transform(fctProbateComplaintSummaryRaw)

print(f"""fctProbateComplaintSummary (type = {type(fctProbateComplaintSummary)} has {fctProbateComplaintSummary.count()} rows with a Schema,
{fctProbateComplaintSummary.schema}""")

StatementMeta(misparkpool, 4907, 39, Finished, Available)

2 schemas in fromSchemas,
fctProbateComplaintSummaryRaw (type = <class 'pyspark.sql.dataframe.DataFrame'>):
StructType(List(StructField(DateKey,TimestampType,true),StructField(OwningLocationKey,DecimalType(10,6),true),StructField(SubjectLocationKey,DecimalType(10,6),true),StructField(CustomerInteractionTypeKey,DecimalType(10,6),true),StructField(InteractionReasonTypeKey,DecimalType(10,6),true),StructField(ChannelMethodTypeKey,DecimalType(10,6),true),StructField(SourceKey,DecimalType(10,6),true),StructField(InteractionCustomerTypeKey,DecimalType(10,6),true),StructField(ReceiptByStepCount,DecimalType(10,6),true),StructField(ReceiptsByItemCount,DecimalType(10,6),true),StructField(ConcludedByStepCount,DecimalType(10,6),true),StructField(ConcludedByItemCount,DecimalType(10,6),true),StructField(ConcludedAtFirstContactCount,DecimalType(6,0),true),StructField(ConcludedInTargetByStepCount,DecimalType(6,0),true),StructField(DaysToConcludeCount,DecimalType(9,0),true),StructField(UpheldCount,Decim

## Write to data lake and DWH

L3 notebook: overwrite all data in the target data lake file and table.

In [56]:
#Counts how many rows will be coppied over when the notebook is run 
RowsCopied += fctProbateComplaintSummary.count()
print(f'RowsCopied = {RowsCopied}')

StatementMeta(misparkpool, 4907, 33, Finished, Available)

RowsCopied = 0

In [63]:
fctProbateComplaintSummary.write.format('delta').mode('overwrite').save(
    DataIO.lakeFilePath(lakeContainerName, lakeAccountName, fctLakeFolder, f'{fctLakeFileName}.parquet')
) #Updating/overwriting the data in https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/fctProbateComplaintSummary_path.png

print('URI = ' + DataIO.lakeFilePath(lakeContainerName, lakeAccountName, fctLakeFolder, f'{fctLakeFileName}.parquet'))

StatementMeta(misparkpool, 4907, 40, Finished, Available)

URI = abfss://transformation@mipolybasestagingtest.dfs.core.windows.net/Layer_3/XCutting/Optic/L3_Optic_fctProbateComplaintSummary.parquet

In [58]:
writer = SynapseDatabaseWriter(dwhEnvironment, dwhDatabase) #Setting up a writer using the class SynapseDatabaseWriter from the notebook Commons/SynapseDatabaseWriter

writer.write(fctProbateComplaintSummary, dwhSchema, dwhTableName) #Writing to the final location, see https://github.com/GuriHMCTS/Demo_L3Optic_ProbateComplaintSummary/blob/main/Images/dwh_location.png

print(f"""In the above cell:
env: {dwhEnvironment}
synapseDatabaseName: {dwhDatabase}

dataframe: {fctProbateComplaintSummary.show(n=2, vertical=True)}
databaseSchema: {dwhSchema}
databaseTable: {dwhTableName}""")

print(type(writer))

StatementMeta(misparkpool, 4907, 35, Finished, Available)


(0 rows)

In the above cell:
env: test
synapseDatabaseName: mi_dwh_test

dataframe: None
databaseSchema: L3_Optic
databaseTable: fctProbateComplaintSummary
<class '__main__.SynapseDatabaseWriter'>

## Construct return value

In [59]:
mssparkutils.notebook.exit(json.dumps(returnValue)) #Exits the notebook, will error as returnValue is not defined 

StatementMeta(misparkpool, 4907, 36, Finished, Available)

NameError: name 'returnValue' is not defined